In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

/home/matevz/.pyenv/versions/3.6.13/envs/hpvm/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [11]:
labels = [
    "Walking",
    "W. Upstairs",
    "W. Downstairs",
    "Sitting",
    "Standing",
    "Lying",
]

segments = []
with open("model_log") as f:
    lines_raw = [l.strip() for l in f.readlines()]
    new_segment = None
    for l in lines_raw:
        if l == "begin":
            new_segment = []
        elif l == "end":
            segments.append(new_segment)
            new_segment = None
        else:
            new_segment.append(l)

df = pd.DataFrame(
    [s.split() for seg in segments for s in seg],
    columns=["conf", "true", "pred", 0, 1, 2, 3, 4, 5],
).astype({
    "conf": "str",
    "true": "int",
    "pred": "int",
    0: "float", 1: "float", 2: "float", 3: "float", 4: "float", 5: "float",
})
del segments

sm_c = df.loc[:, [0, 1, 2, 3, 4, 5]]
df["min"] = sm_c.min(axis=1)
df["max"] = sm_c.max(axis=1)


def max_2_difference(row: pd.Series):
    max2 = row.nlargest(2).values
    return max2[0] - max2[1]


df["max-max2"] = sm_c.apply(max_2_difference, axis=1, result_type='expand')
df["var"] = sm_c.var(axis=1)
df["max-min"] = df["max"]-df["min"]
df["correct"] = (df["true"] == df["pred"]).astype(int)

accuracies = df.groupby(["conf", "true"]).agg({'correct': 'sum', 'pred': 'count'})
accuracies["acc"] = accuracies["correct"] / accuracies["pred"]

accuracies_pred = df.groupby(["conf", "pred"]).agg({'correct': 'sum', 'true': 'count'})
accuracies_pred["acc"] = accuracies_pred["correct"] / accuracies_pred["true"]

# print("before")
# print(df["correct"].value_counts().sort_index())

# print("after")
# original = df.copy()
# for _ in range(5):
#     df = df.append(original[original.correct == 0])
# print(df["correct"].value_counts().sort_index())

In [3]:
feats = ["max", "max-max2"]
clz = "correct"

In [ ]:
def do():
    from sklearn.model_selection import train_test_split, cross_validate
    from sklearn.naive_bayes import GaussianNB, MultinomialNB
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, roc_auc_score, plot_roc_curve, f1_score
    from matplotlib.lines import Line2D

    nlabels = len(df["true"].unique())
    nconfs = len(df["conf"].unique())

    fig, axs = plt.subplots(nrows=nconfs, ncols=nlabels, figsize=(6, 10), sharex=True, sharey=True)
    for i, (conf, grpc) in enumerate(df.groupby(["conf"])):
        for j, (label, grp) in enumerate(grpc.groupby(["pred"])):

            ax = axs[i, label]

            X, y = grp.loc[:, feats], grp.loc[:, clz]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            acc = accuracies_pred.loc[conf].loc[label].acc

            ax.set_ylim(-0.05, 1.05)
            ax.hlines(acc, 0, 1, colors='r', linestyles='--')

            classifier = GaussianNB()

            classifier.fit(X_train, y_train)

            y_pred = classifier.predict(X_test)

            plot_kw = dict(c='b')

            def text(s): return ax.text(0.5, 0.5, s, ha='center', va='center', fontsize='small')

            if np.all(y_pred == 0):
                # TPR = 0
                # TNR = 1
                ax.set_facecolor((0.9, 0.9, 0.9))
                text("TPR=0\nTNR=1")
            elif np.all(y_pred == 1):
                # TPR = 1
                # TNR = 0
                ax.set_facecolor((0.9, 0.9, 0.9))
                text("TPR=1\nTNR=0")
            elif np.any(y_test == 0) and np.any(y_test == 1):
                # else:
                # dsp = plot_roc_curve(classifier, X_test, y_test, ax=ax, pos_label=1, name=f"Predicting correct")
                dsp = plot_roc_curve(classifier, X_test, y_test, ax=ax, pos_label=0,
                                    name=f"Predicting incorrect", **plot_kw)
                ax.plot([0, 1], [0, 1], '--', c='gray')
                ax.legend().remove()
            else:
                text("No data")
                ax.set_facecolor((0.9, 0.9, 0.9))

            ax.set_ylabel("")
            ax.set_xlabel("")

            if label == len(labels)-1:
                ax.text(1.2, 0.5, conf)

            if i == 0:
                ax.set_title(f"{labels[label]}", rotation=45, fontsize='small')

    fig.text(0.5, 0.98, "Predicted activity", ha='center')  # x

    fig.text(0.5, 0.07, "True negative rate (TNR)", ha='center')  # x
    fig.text(0.04, 0.5, "True positive rate (TPR)", va='center', rotation='vertical')  # y

    legend_elements = [Line2D([0], [0], c='b', ls='-', label='ROC curve'),
                    Line2D([0], [0], c='r', ls='--', label='HAR model accuracy'), ]
    fig.legend(handles=legend_elements, ncol=2, loc='center', bbox_to_anchor=(0, 0.03, 1, 0))


    plt.savefig("roc-confidence-bayes.pdf", bbox_inches='tight')
    plt.show()

do()


In [19]:
def do():
    from sklearn.model_selection import train_test_split, cross_validate
    from sklearn.naive_bayes import GaussianNB, MultinomialNB
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, roc_auc_score, plot_roc_curve, f1_score, make_scorer, precision_score, recall_score
    from matplotlib.lines import Line2D
    from sys import stderr
    from collections import Counter

    nlabels = len(df["true"].unique())
    nconfs = len(df["conf"].unique())

    score_grid_p = np.zeros((nconfs, nlabels))
    score_grid_r = np.zeros((nconfs, nlabels))
    score_grid_har = np.zeros((nconfs, nlabels))

    def for_pos_label(pos_label):
        precision_scorer = make_scorer(precision_score, pos_label=pos_label, zero_division=0)
        recall_scorer = make_scorer(recall_score, pos_label=pos_label, zero_division=0)

        for i, (conf, grpc) in enumerate(df.groupby(["conf"])):
            for j, (label, grp) in enumerate(grpc.groupby(["pred"])):

                X, y = grp.loc[:, feats], grp.loc[:, clz]

                score_grid_har[i, j] = accuracies_pred.loc[conf].loc[label].acc

                cv=3

                counts = Counter(y)
                if counts[0] <= cv or counts[1] <= cv:
                    score_grid_p[i, j] = np.nan
                    score_grid_r[i, j] = np.nan
                    continue

                classifier = GaussianNB()

                scores = cross_validate(classifier, X, y, scoring={
                                        'p': precision_scorer, 'r': recall_scorer},cv=cv)

                score_grid_p[i, j] = scores["test_p"].mean()
                score_grid_r[i, j] = scores["test_r"].mean()

        f1_avg = 2*(score_grid_p*score_grid_r)/(score_grid_p+score_grid_r)

        fig, axs = plt.subplots(1, 4, sharey=True)

        plt.setp(axs,
                xticks=np.arange(nlabels), xticklabels=[l.replace("\n", " ") for l in labels],
                yticks=np.arange(nconfs))

        for ax in axs:
            plt.setp(ax.get_xticklabels(), rotation='vertical')

        cmap = "viridis"

        axs[0].set_title("Precision")
        axs[0].imshow(score_grid_p, vmin=0, vmax=1, cmap=cmap)

        axs[1].set_title("Recall")
        axs[1].imshow(score_grid_r, vmin=0, vmax=1, cmap=cmap)

        axs[2].set_title("F1")
        axs[2].imshow(f1_avg, vmin=0, vmax=1, cmap=cmap)

        axs[3].set_title("Baseline F1\nRecall=1")
        baseline = (1-score_grid_har) if pos_label == 0 else score_grid_har
        im = axs[3].imshow(baseline, vmin=0, vmax=1, cmap=cmap)



        (_, y0, _, hig) = axs[2].get_position().bounds

        # fig.subplots_adjust(right=0.8)
        # put colorbar at desire position
        cbar_ax = fig.add_axes([0.95, y0, 0.02, hig])
        fig.colorbar(im, cax=cbar_ax)

        fig.text(0.5, -0.14, 'Predicted activity', ha='center')
        fig.text(0.04, 0.5, 'Configuration index', va='center', rotation='vertical')
        # fig.suptitle(f"Correctness prediction for class: {'correct' if pos_label else 'wrong'}")

        plt.savefig(f"grid-confidence-bayes-{pos_label}.pdf", bbox_inches='tight')
        plt.show()
    for_pos_label(0)
    for_pos_label(1)
    

do()


In [ ]:
def do():
    from sklearn.model_selection import train_test_split, cross_validate
    from sklearn.naive_bayes import GaussianNB, MultinomialNB
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, roc_auc_score, plot_roc_curve, f1_score, make_scorer, precision_score, recall_score
    from matplotlib.lines import Line2D
    from sys import stderr
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    nlabels = len(df["true"].unique())
    nconfs = len(df["conf"].unique())

    score_grid_acc = np.zeros((nlabels, nconfs))
    score_grid_qos = np.zeros((nlabels, nconfs))

    for i, (conf, grpc) in enumerate(df.groupby(["conf"])):
        for j, (label, grp) in enumerate(grpc.groupby(["true"])):
            acc = accuracies.loc[conf].loc[label].acc

            score_grid_acc[j, i] = acc
            score_grid_qos[j, i] = max(accuracies.loc["conf0"].loc[label].acc - acc, 0)

    fig, axs = plt.subplots(1, 2, figsize=(8,4),sharey=True)

    plt.setp(axs,
             yticks=np.arange(nlabels), yticklabels=[l.replace("\n", " ") for l in labels],
             xticks=np.arange(nconfs), xticklabels=np.arange(nconfs))

    # for ax in axs:
    #     plt.setp(ax.get_xticklabels(), rotation='vertical')

    axs[0].set_title("Accuracy")
    axs[0].set_xlabel("Configuration index")
    im0 = axs[0].imshow(score_grid_acc, vmin=0, vmax=1, cmap="Blues_r")
    divider0 = make_axes_locatable(axs[0])
    cax0 = divider0.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(im0, cax=cax0)

    axs[1].set_title("QoS loss")
    axs[1].set_xlabel("Configuration index")
    im1 = axs[1].imshow(score_grid_qos, cmap="Greens")
    divider1 = make_axes_locatable(axs[1])
    cax1 = divider1.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(im1, cax=cax1)

    plt.savefig("grid-accuracy-and-qos.pdf", bbox_inches='tight')
    plt.show()


do()
